# Install numba

In [1]:
!pip  install numba

     ---------------------------------------- 2.6/2.6 MB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 28.1/28.1 MB 9.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: C:\Users\hieuq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Using numba to get the information of the device

In [2]:
import numba
from numba import cuda

In [3]:
cuda.detect()

Found 1 CUDA devices
id 0    b'GeForce GTX 1050 Ti'                              [SUPPORTED]
                      Compute Capability: 6.1
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-6d8e1ed7-4b24-5716-1782-904e5457d239
                                Watchdog: Enabled
                            Compute Mode: WDDM
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [4]:
device = cuda.select_device(0)

In [5]:
cuda.gpus

In [11]:
getattr(device, 'MULTIPROCESSOR_COUNT')

6

In [12]:
device.compute_capability

(6, 1)

In [23]:
dir(device)

['ASYNC_ENGINE_COUNT',
 'CAN_MAP_HOST_MEMORY',
 'CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM',
 'COMPUTE_CAPABILITY_MAJOR',
 'COMPUTE_CAPABILITY_MINOR',
 'KERNEL_EXEC_TIMEOUT',
 'MULTIPROCESSOR_COUNT',
 'PCI_BUS_ID',
 'PCI_DEVICE_ID',
 'SINGLE_TO_DOUBLE_PRECISION_PERF_RATIO',
 'TCC_DRIVER',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'attributes',
 'compute_capability',
 'from_identity',
 'get_device_identity',
 'get_primary_context',
 'id',
 'name',
 'primary_context',
 'release_primary_context',
 'reset',
 'supports_float16',
 'uuid']

In [15]:
attribs = [s for s in dir(device) if s.isupper()]
for attr in attribs:
    print(attr, '=', getattr(device, attr))

COMPUTE_CAPABILITY_MAJOR = 6
COMPUTE_CAPABILITY_MINOR = 1
KERNEL_EXEC_TIMEOUT = 1
MULTIPROCESSOR_COUNT = 6
PCI_BUS_ID = 1
PCI_DEVICE_ID = 0
SINGLE_TO_DOUBLE_PRECISION_PERF_RATIO = 32
TCC_DRIVER = 0


In [6]:
cc_cores_per_SM_dict = {
    (2,0) : 32,
    (2,1) : 48,
    (3,0) : 192,
    (3,5) : 192,
    (3,7) : 192,
    (5,0) : 128,
    (5,2) : 128,
    (6,0) : 64,
    (6,1) : 128,
    (7,0) : 64,
    (7,5) : 64,
    (8,0) : 64,
    (8,6) : 128,
    (8,9) : 128,
    (9,0) : 128
    }
# the above dictionary should result in a value of "None" if a cc match 
# is not found.  The dictionary needs to be extended as new devices become
# available, and currently does not account for all Jetson devices
device = cuda.get_current_device()
my_sms = getattr(device, 'MULTIPROCESSOR_COUNT')
my_cc = device.compute_capability
cores_per_sm = cc_cores_per_SM_dict.get(my_cc)
total_cores = cores_per_sm*my_sms
print("GPU compute capability: " , my_cc)
print("GPU total number of SMs: " , my_sms)
print("total cores: " , total_cores)

GPU compute capability:  (6, 1)
GPU total number of SMs:  6
total cores:  768


# Get memory information

In [18]:
cuda.current_context().get_memory_info()

MemoryInfo(free=3543112091, total=4294967296)

In [6]:
!nvidia-smi

Sat Oct 21 11:38:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 431.84       Driver Version: 431.84       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 105... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8    N/A /  N/A |     75MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  